# 保存模型

[![](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/docs/notebook/mindspore_save_model.ipynb)

## 概述

在模型训练过程中，可以添加检查点(CheckPoint)用于保存模型的参数，以便进行推理及再训练使用。如果想继续在不同硬件平台上做推理，可通过网络和CheckPoint格式文件生成对应的MindIR、AIR和ONNX格式文件。

- MindIR：MindSpore的一种基于图表示的函数式IR，其最核心的目的是服务于自动微分变换，目前可用于MindSpore Lite端侧推理。

- CheckPoint：MindSpore的存储了所有训练参数值的二进制文件。采用了Google的Protocol Buffers机制，与开发语言、平台无关，具有良好的可扩展性。CheckPoint的protocol格式定义`在mindspore/ccsrc/utils/checkpoint.proto`中。

- AIR：全称Ascend Intermediate Representation，类似ONNX，是华为定义的针对机器学习所设计的开放式的文件格式，能更好地适配Ascend AI处理器。

- ONNX：全称Open Neural Network Exchange，是一种针对机器学习所设计的开放式的文件格式，用于存储训练好的模型。

以下通过图片分类应用示例来介绍保存CheckPoint格式文件和导出MindIR、AIR和ONNX格式文件的方法。

> 本文档适用于CPU、GPU和Ascend AI处理器环境。

说明：<br/>在保存和转换模型前，我们需要完整进行图片分类训练，包含数据准备、定义网络、定义损失函数及优化器和训练网络，详细信息可以参考[初学入门](https://www.mindspore.cn/tutorials/zh-CN/master/quick_start.html)。

整体流程如下：

1. 数据准备

2. 构造神经网络

3. 搭建训练网络、定义损失函数及优化器

4. 保存CheckPoint格式文件

5. 导出不同格式文件

## 数据准备

### 下载MNIST数据集

以下示例代码将数据集下载并解压到指定位置。

In [ ]:
import os
import requests

requests.packages.urllib3.disable_warnings()

def download_dataset(dataset_url, path):
    filename = dataset_url.split("/")[-1]
    save_path = os.path.join(path, filename)
    if os.path.exists(save_path):
        return
    if not os.path.exists(path):
        os.makedirs(path)
    res = requests.get(dataset_url, stream=True, verify=False)
    with open(save_path, "wb") as f:
        for chunk in res.iter_content(chunk_size=512):
            if chunk:
                f.write(chunk)
    print("The {} file is downloaded and saved in the path {} after processing".format(os.path.basename(dataset_url), target_path))

train_path = "datasets/MNIST_Data/train"
test_path = "datasets/MNIST_Data/test"

download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-labels-idx1-ubyte", train_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-images-idx3-ubyte", train_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-labels-idx1-ubyte", test_path)
download_dataset("https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-images-idx3-ubyte", test_path)

下载的数据集文件的目录结构如下：

```text
./datasets/MNIST_Data
├── test
│   ├── t10k-images-idx3-ubyte
│   └── t10k-labels-idx1-ubyte
└── train
    ├── train-images-idx3-ubyte
    └── train-labels-idx1-ubyte
```

### 数据处理：

数据集对于训练非常重要，好的数据集可以有效提高训练精度和效率，在加载数据集前，我们通常会对数据集进行一些处理。

#### 定义数据集及数据操作

我们定义一个函数`create_dataset`来创建数据集。在这个函数中，我们定义好需要进行的数据增强和处理操作：

1. 定义数据集。
2. 定义进行数据增强和处理所需要的一些参数。
3. 根据参数，生成对应的数据增强操作。
4. 使用`map`映射函数，将数据操作应用到数据集。
5. 对生成的数据集进行处理。

In [2]:
import mindspore.dataset.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.dataset.vision import Inter
from mindspore import dtype as mstype
import mindspore.dataset as ds


def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    """
    create dataset for train or test

    Args:
        data_path (str): Data path
        batch_size (int): The number of data records in each group
        repeat_size (int): The number of replicated data records
        num_parallel_workers (int): The number of parallel workers
    """
    # define dataset
    mnist_ds = ds.MnistDataset(data_path)

    # define some parameters needed for data enhancement and rough justification
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # according to the parameters, generate the corresponding data enhancement method
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml)
    rescale_op = CV.Rescale(rescale, shift)
    hwc2chw_op = CV.HWC2CHW()
    type_cast_op = C.TypeCast(mstype.int32)

    # using map to apply operations to a dataset
    mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=resize_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=rescale_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=rescale_nml_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=hwc2chw_op, input_columns="image", num_parallel_workers=num_parallel_workers)

    # process the generated dataset
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds


## 构造神经网络

在对手写字体识别上，通常采用卷积神经网络架构（CNN）进行学习预测，最经典的属1998年由Yann LeCun创建的LeNet5架构，在构建LeNet5前，我们需要对全连接层以及卷积层进行初始化。

In [3]:
import mindspore.nn as nn
from mindspore.common.initializer import Normal

class LeNet5(nn.Cell):
    """Lenet network structure."""
    # define the operator required
    def __init__(self, num_class=10, num_channel=1):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    # use the preceding operators to construct networks
    def construct(self, x):
        x = self.max_pool2d(self.relu(self.conv1(x)))
        x = self.max_pool2d(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 搭建训练网络

构建完成神经网络后，就可以着手进行训练网络的构建，包括定义损失函数及优化器。

In [4]:
import os
from mindspore.nn import SoftmaxCrossEntropyWithLogits
from mindspore.nn import Accuracy
from mindspore import context, Model

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

lr = 0.01
momentum = 0.9

# create the network
network = LeNet5()

# define the optimizer
net_opt = nn.Momentum(network.trainable_params(), lr, momentum)

# define the loss function
net_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

# define the model
model = Model(network, net_loss, net_opt, metrics={"Accuracy": Accuracy()})

epoch_size = 1
mnist_path = "./datasets/MNIST_Data"

eval_dataset = create_dataset("./datasets/MNIST_Data/test")

repeat_size = 1
print("========== The Training Model is Defined. ==========")


========== The Training Model is Defined. ==========


## 保存CheckPoint格式文件

在模型训练的过程中，使用Callback机制传入回调函数`ModelCheckpoint`对象，可以保存模型参数，生成CheckPoint文件。

通过`CheckpointConfig`对象可以设置CheckPoint的保存策略。保存的参数分为网络参数和优化器参数。

`ModelCheckpoint`提供默认配置策略，方便用户快速上手，用户可以根据具体需求对`CheckPoint`策略进行配置。

### 单次运行训练脚本保存模型

在这里配置`CheckPoint`时，设置的是每隔375个steps就保存一次，最多保留10个CheckPoint文件，生成前缀名为“lenet”，具体用法如下：

In [5]:
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

model_path = './models/ckpt/mindspore_save_model/'
# clean up old run files before in Linux
os.system('rm -f {0}*.ckpt {0}*.meta {0}*.pb'.format(model_path))

# define config_ck for specifying the steps to save the checkpoint and the maximum file numbers
config_ck = CheckpointConfig(save_checkpoint_steps=375, keep_checkpoint_max=10)
# define ckpoint_cb for specifying the prefix of the file and the saving directory
ckpoint_cb = ModelCheckpoint(prefix='lenet', directory=model_path, config=config_ck)
#load the training dataset
ds_train = create_dataset(os.path.join(mnist_path, "train"), 32, repeat_size)
print("========== The Training is Starting. ==========")
model.train(epoch_size, ds_train, callbacks=ckpoint_cb, dataset_sink_mode=False)
print("========== The Training is Completed and the Checkpoint Files are Saved. ==========")

========== The Training is Starting. ==========
========== The Training is Completed and the Checkpoint Files are Saved. ==========


上述代码中，首先需要初始化一个CheckpointConfig类对象，用来设置保存策略。

- `save_checkpoint_steps`表示每隔多少个step保存一次。
- `keep_checkpoint_max`表示最多保留CheckPoint文件的数量。
- `prefix`表示生成CheckPoint文件的前缀名。
- `directory`表示存放文件的目录。
- `epoch_size`表示每个epoch需要遍历完成图片的batch数。
- `ds_train`表示数据集。

创建一个ModelCheckpoint对象把它传递给`model.train`方法，就可以在训练过程中使用CheckPoint功能了。

生成的CheckPoint文件如下：

```text
./models/ckpt/mindspore_save_model
├── lenet-1_1125.ckpt
├── lenet-1_1500.ckpt
├── lenet-1_1875.ckpt
├── lenet-1_375.ckpt
├── lenet-1_750.ckpt
└── lenet-graph.meta
```

其中：

- `lenet-graph.meta`为编译后的计算图。
- CheckPoint文件后缀名为`.ckpt`，文件的命名方式表示保存参数所在的epoch和step数。
- `lenet-1_750.ckpt`表示保存的是第1个epoch的第750个step的模型参数。

### 多次运行训练脚本保存模型

如果用户使用相同的前缀名，运行多次训练脚本，可能会生成同名CheckPoint文件。MindSpore为方便用户区分每次生成的文件，会在用户定义的前缀后添加_和数字加以区分。如下所示：

In [7]:
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig
import os

# clean up old run files before in Linux
os.system('rm -f {}lenet_2*.ckpt'.format(model_path))

config_ck = CheckpointConfig(save_checkpoint_steps=375, keep_checkpoint_max=10)
# Specify that here the script is executed for the second time
ckpoint_cb = ModelCheckpoint(prefix='lenet_2', directory='./models/ckpt/mindspore_save_model', config=config_ck)
ds_train = create_dataset(os.path.join(mnist_path, "train"), 32, repeat_size)
print("========== The Training is Starting. ==========")
model.train(epoch_size, ds_train, callbacks=ckpoint_cb, dataset_sink_mode=False)
print("========== The Training is Completed and the Checkpoint Files are Saved. ==========")

========== The Training is Starting. ==========
========== The Training is Completed and the Checkpoint Files are Saved. ==========


生成的CheckPoint文件（以`lenet-2`为前缀的`.ckpt`文件）如下：

```text
./models/ckpt/mindspore_save_model
├── lenet-1_1125.ckpt
├── lenet-1_1500.ckpt
├── lenet-1_1875.ckpt
├── lenet-1_375.ckpt
├── lenet-1_750.ckpt
├── lenet_2-1_1125.ckpt
├── lenet_2-1_1500.ckpt
├── lenet_2-1_1875.ckpt
├── lenet_2-1_375.ckpt
├── lenet_2-1_750.ckpt
├── lenet_2-graph.meta
└── lenet-graph.meta
```

lenet_2-1_750.ckpt 表示本次运行脚本生成的第1个epoch的第750个step的CheckPoint文件。

### 配置时间策略保存模型

MindSpore提供了两种保存CheckPoint策略：迭代策略和时间策略，上述代码为迭代策略。我们可以通过创建`CheckpointConfig`对象设置相应策略，CheckpointConfig中共有四个参数可以设置：

- `save_checkpoint_steps`：表示每隔多少个step保存一个CheckPoint文件，默认值为1。

- `save_checkpoint_seconds`：表示每隔多少秒保存一个CheckPoint文件，默认值为0。

- `keep_checkpoint_max`：表示最多保存多少个CheckPoint文件，默认值为5。

- `keep_checkpoint_per_n_minutes`：表示每隔多少分钟保留一个CheckPoint文件，默认值为0。

因为时间策略需要训练时间稍长一些，所以这里把`epoch_size`改为10。

以下代码为时间策略：

In [9]:
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig
import os

os.system('rm -f {}lenet_3*.ckpt'.format(model_path))
# define config_ck for specifying the seconds to save the checkpoint and the maximum file numbers
config_ck = CheckpointConfig(save_checkpoint_steps=None, save_checkpoint_seconds=10, keep_checkpoint_max=None, keep_checkpoint_per_n_minutes=1)
# define ckpoint_cb for specifying the prefix of the file and the saving directory
ckpoint_cb = ModelCheckpoint(prefix='lenet_3', directory='./models/ckpt/mindspore_save_model', config=config_ck)
#load the training dataset
epoch_size = 2
ds_train = create_dataset(os.path.join(mnist_path, "train"), 32, repeat_size)
print("========== The Training is Starting. ==========")
model.train(epoch_size, ds_train, callbacks=ckpoint_cb, dataset_sink_mode=False)
print("========== The Training is Completed and the Checkpoint Files are Saved. ==========")

========== The Training is Starting. ==========
========== The Training is Completed and the Checkpoint Files are Saved. ==========


这里`save_checkpoint_seconds`和`keep_checkpoint_per_n_minutes`这两个分别设置为10和1。<br>
意思是每10秒会保存一个CheckPoint文件，每隔1分钟会保留一个CheckPoint文件。假设训练持续了1分钟，那总共会生成7个CheckPoint文件，但是当训练结束后，实际上会看到4个CheckPoint文件（以`lenet-3`为前缀的`.ckpt`文件），即保存下来的3个文件和默认保存最后一个step的CheckPoint文件。

生成的CheckPoint文件如下：

```text
./models/ckpt/mindspore_save_model
├── lenet-1_1125.ckpt
├── lenet-1_1500.ckpt
├── lenet-1_1875.ckpt
├── lenet-1_375.ckpt
├── lenet-1_750.ckpt
├── lenet_2-1_1125.ckpt
├── lenet_2-1_1500.ckpt
├── lenet_2-1_1875.ckpt
├── lenet_2-1_375.ckpt
├── lenet_2-1_750.ckpt
├── lenet_2-graph.meta
├── lenet_3-1_1023.ckpt
├── lenet_3-2_1254.ckpt
├── lenet_3-2_1875.ckpt
├── lenet_3-2_194.ckpt
├── lenet_3-graph.meta
└── lenet-graph.meta
```

另请注意，如果想要删除.ckpt文件时，请同步删除.meta 文件。

## 导出MindIR格式文件

当有了CheckPoint文件后，如果想继续在MindSpore Lite端侧做推理，需要通过网络和CheckPoint生成对应的MindIR格式模型文件。当前支持基于静态图，且不包含控制流语义的推理网络导出。建议使用`.mindir`作为MINDIR格式文件的后缀名。导出该格式文件的代码如下：

In [11]:
from mindspore import export, load_checkpoint, load_param_into_net
from mindspore import Tensor
import numpy as np

lenet = LeNet5()
# return a parameter dict for model
param_dict = load_checkpoint("./models/ckpt/mindspore_save_model/lenet-1_1875.ckpt")
# load the parameter into net
load_param_into_net(lenet, param_dict)
input = np.random.uniform(0.0, 1.0, size=[32, 1, 32, 32]).astype(np.float32)
# export the file with the specified name and format
export(lenet, Tensor(input), file_name='lenet-1_1875', file_format='MINDIR',)

完成以后，在当前目录下会生成一个MindIR格式文件，文件名为：`lenet-1_1875.mindir`。

## 导出ONNX格式文件

当有了CheckPoint文件后，如果想继续在Ascend AI处理器、GPU、CPU等多种硬件上做推理，需要通过网络和CheckPoint生成对应的ONNX格式模型文件，建议使用`.onnx`作为ONNX格式文件的后缀名。导出该格式文件的代码样例如下：

In [12]:
from mindspore import export, load_checkpoint, load_param_into_net
from mindspore import Tensor
import numpy as np
lenet = LeNet5()
# return a parameter dict for model
param_dict = load_checkpoint("./models/ckpt/mindspore_save_model/lenet-1_1875.ckpt")
# load the parameter into net
load_param_into_net(lenet, param_dict)
input = np.random.uniform(0.0, 1.0, size=[32, 1, 32, 32]).astype(np.float32)
# export the file with the specified name and format
export(lenet, Tensor(input), file_name='lenet-1_1875', file_format='ONNX')

完成以后，在当前目录下会生成一个ONNX格式文件，文件名为：`lenet-1_1875.onnx`。

## 总结

以上就是保存模型并导出文件的全部体验过程，我们通过本次体验全面了解了训练模型的保存以及如何导出成为不同格式的文件，以便用于不同平台上的推理。